In [4]:
%pip install pypdf
%pip install joblib
%pip install pandas
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to us

In [18]:
from pypdf import PdfReader
import re
import pandas as pd

# Data extraction
parties = ['liberal', 'conservative', 'ndp', 'green']

not_alpha_regex = r"[^a-zA-Z]"
not_main_chars_regex = r"[^a-zA-Z\s.]"
excess_spaces_regex = r"\s+"

party_labels = []
texts = []

def hasAlphabeticalChars(string):
    return len(re.sub(not_alpha_regex, "", string)) > 0

for i, party in enumerate(parties):
    reader = PdfReader(f"../data/platforms/{party}.pdf")

    for page in reader.pages:
        raw_text = page.extract_text()
        cleaned_text = re.sub(not_main_chars_regex, "", raw_text)
        cleaned_text = re.sub(excess_spaces_regex, " ", cleaned_text)
        
        page_sentences = list(filter(hasAlphabeticalChars, cleaned_text.split('.')))
        
        # Appending sentences to list
        for passage in page_sentences:
            party_labels.append(i)
            texts.append(passage)

sentences_data = {'text': texts, 'label': party_labels}

sentences = pd.DataFrame(sentences_data)

In [6]:
# Example data

# Load Canadian political dataset (Replace with actual data)
data = pd.DataFrame({
    "text": [
        "The government must take action on climate change immediately.",
        "We need lower taxes and smaller government.",
        "Healthcare should be publicly funded and accessible to all.",
        "Quebec deserves greater autonomy in national matters.",
        "We must reduce immigration to protect Canadian culture."
    ],
    "label": [0, 1, 2, 3, 4]  # Liberal, Conservative, NDP, BQ, PPC
})


In [21]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


sentences
data

# Split dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    sentences["text"].tolist(), sentences["label"].tolist(), test_size=0.2, random_state=42
)

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels}).map(tokenize_function, batched=True)
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels}).map(tokenize_function, batched=True)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    auto_find_batch_size=True,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
)

# Train the model
trainer.train()

# Save model
model.save_pretrained("./canadian_political_bert")
tokenizer.save_pretrained("./canadian_political_bert")

print("Model training complete and saved.")


Map: 100%|██████████| 1390/1390 [00:00<00:00, 3460.01 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/isss11/.local/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_6449/1004572970.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.984900,0.738537


Model training complete and saved.


In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

def predict_political_affiliation(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move to GPU if available

    with torch.no_grad():
        print(inputs)
        
        outputs = model(**inputs)
    
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    # Map class index to party
    return parties[predicted_class]

print(predict_political_affiliation("Canada’s recovery must be strong, inclusive, and focused on building a better future. That’s why we’re investing in childcare, expanding healthcare access, and taking decisive action on climate change. Our plan will help families, create good-paying jobs, and ensure economic growth that benefits everyone. A Liberal government will continue to support the middle class while making Canada a global leader in innovation and sustainability."))


{'input_ids': tensor([[  101,  2710,  1521,  1055,  7233,  2442,  2022,  2844,  1010, 18678,
          1010,  1998,  4208,  2006,  2311,  1037,  2488,  2925,  1012,  2008,
          1521,  1055,  2339,  2057,  1521,  2128, 19920,  1999,  2775, 16302,
          1010,  9186,  9871,  3229,  1010,  1998,  2635, 13079,  2895,  2006,
          4785,  2689,  1012,  2256,  2933,  2097,  2393,  2945,  1010,  3443,
          2204,  1011,  7079,  5841,  1010,  1998,  5676,  3171,  3930,  2008,
          6666,  3071,  1012,  1037,  4314,  2231,  2097,  3613,  2000,  2490,
          1996,  2690,  2465,  2096,  2437,  2710,  1037,  3795,  3003,  1999,
          8144,  1998, 15169,  1012,   102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0,